In [ ]:
# --- FOR DATA AUGMENTATION ---

In [15]:
%%capture
%run ./data_process.ipynb

In [ ]:
# Define folders for augmentation dataset
import os
import cv2
import numpy as np
import random
from pathlib import Path

# ~~~~~~~~~~ Config ~~~~~~~~~~
IMG_DIR   = r"C:\Users\youch\CPP_AVL_ObjDet\images\train"
LABEL_DIR = r"C:\Users\youch\CPP_AVL_ObjDet\labels\train"
ROOT = r"C:\Users\youch\CPP_AVL_ObjDet"
OUT_IMG = os.path.join(ROOT, "images", "train_augmented")
OUT_LBL = os.path.join(ROOT, "labels", "train_augmented")

MIN_BOX_AREA   = 0.001    # discard boxes smaller than 0.1% of image after transform

os.makedirs(OUT_IMG, exist_ok=True)
os.makedirs(OUT_LBL, exist_ok=True)         


# ~~~~~~~~~~~~~~~ Function to load labels ~~~~~~~~~~~~~~~~~~~~~~
def load_labels(label_path):
    labels = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])
                labels.append((class_id, x_center, y_center, width, height))
    return labels

# ~~~~~~~~~~~~ Save the labels into the output folder ~~~~~~~~~~~~
def save_labels(labels, out_path):
    with open(out_path, "w") as f:
        for b in labels:
            f.write(f"{b[0]} {b[1]:.6f} {b[2]:.6f} {b[3]:.6f} {b[4]:.6f}\n")

# ~~~~~~~~~~~~ Clipping ~~~~~~~~~~~~
def clip_labels(cx, cy, w, h):
    x1 = max(0.0, cx - w / 2)
    y1 = max(0.0, cy - h / 2)
    x2 = min(1.0, cx + w / 2)
    y2 = min(1.0, cy + h / 2)
    new_w = x2 - x1
    new_h = y2 - y1
    new_cx = x1 + new_w / 2
    new_cy = y1 + new_h / 2
    return new_cx, new_cy, new_w, new_h


# ~~~~~~~~~~~ Check if it is a valid box after transformation ~~~~~~~~~~~
def is_valid_box(cx, cy, w, h):
    return w > 0 and h > 0 and w * h >= MIN_BOX_AREA


# ~~~~~~~~~~~ Augmentation #1: Cropping ~~~~~~~~~~~~
def random_crop(img, boxes, target_classes = {1, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14}, crop_ratio_range=(0.6, 0.9)):
    has_target = any(b[0] in target_classes for b in boxes)
    if not has_target:
        return None, None
    
    h, w = img.shape[:2]
    ratio = random.uniform(*crop_ratio_range)
    
    crop_w = int(w * ratio)
    crop_h = int(h * ratio)
    x_off = random.randint(0, w - crop_w)
    y_off = random.randint(0, h - crop_h)
    
    cropped = img[y_off:y_off+crop_h, x_off:x_off+crop_w]
    cropped = cv2.resize(cropped, (w, h))  # Resize back to original size
    
    new_boxes = []
    
    for cls, cx, cy, bw, bh in boxes:
        # Convert to pixel coordinates
        px1 = (cx - bw / 2) * w
        py1 = (cy - bh / 2) * h
        px2 = (cx + bw / 2) * w
        py2 = (cy + bh / 2) * h
        
        # Clip to crop
        cx1 = max(px1, x_off)
        cy1 = max(py1, y_off)
        cx2 = min(px2, x_off + crop_w)
        cy2 = min(py2, y_off + crop_h)
        
        if cx2 <= cx1 or cy2 <= cy1:
            continue  # Box is completely outside crop
        
        
        visible_area = (cx2 - cx1) * (cy2 - cy1)
        original_area = (px2 - px1) * (py2 - py1)
        if original_area > 0 and visible_area / original_area < 0.5:
            continue  # Less than 50% visible, discard
        
        
        # Normalize back to [0,1]
        new_cx = (cx1 + cx2) / 2 / crop_w
        new_cy = (cy1 + cy2) / 2 / crop_h
        new_bw = (cx2 - cx1) / crop_w
        new_bh = (cy2 - cy1) / crop_h
        
        if is_valid_box(new_cx, new_cy, new_bw, new_bh):
            new_boxes.append([cls, new_cx, new_cy, new_bw, new_bh])
            
    return cropped, new_boxes

# ~~~~~~~~~~~ Augmentation #2: Horizontal Flip ~~~~~~~~~~~~
def horizontal_flip(img, boxes, target_classes={1, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14}):
    has_target = any(b[0] in target_classes for b in boxes)
    if not has_target:
        return None, None
    
    flipped = cv2.flip(img, 1)
    new_boxes = []
    for cls, cx, cy, bw, bh in boxes:
        new_boxes.append([cls, 1.0 - cx, cy, bw, bh])  # mirror cx
    
    return flipped, new_boxes

# ~~~~~~~~~~~ Augmentation #3: Cropped Person ~~~~~~~~~~~~
def cropped_person(img, boxes):
    h, w = img.shape[:2]
    person_boxes = [b for b in boxes if b[0] == 0]  # class 0 = person
    if not person_boxes:
        return None, None
    
    #  Pick a random person to crop around
    target = random.choice(person_boxes)
    cls, cx, cy, bw, bh = target
    
    # Convert to pixel coords
    px1 = int((cx - bw/2) * w)
    py1 = int((cy - bh/2) * h)
    px2 = int((cx + bw/2) * w)
    py2 = int((cy + bh/2) * h)
    person_h = py2 - py1
    
    # Randomly crop top half or bottom half
    if random.random() < 0.5:
        # Keep top half (head to waist)
        crop_y2 = py1 + int(person_h * random.uniform(0.4, 0.6))
        crop_y1 = 0
    else:
        # Keep bottom half (waist to feet)
        crop_y1 = py1 + int(person_h * random.uniform(0.4, 0.6))
        crop_y2 = h
    
    crop_y1 = max(0, crop_y1)
    crop_y2 = min(h, crop_y2)
    
    if crop_y2 - crop_y1 < 50:
        return None, None
    
    cropped = img[crop_y1:crop_y2, 0:w]
    new_h = crop_y2 - crop_y1
    cropped = cv2.resize(cropped, (w, h))  # resize back to original dims
    
    new_boxes = []
    for cls, cx, cy, bw, bh in boxes:
        px1 = (cx - bw/2) * w;  py1 = (cy - bh/2) * h
        px2 = (cx + bw/2) * w;  py2 = (cy + bh/2) * h
        
        # Intersect with crop
        iy1 = max(py1, crop_y1);  iy2 = min(py2, crop_y2)
        ix1 = max(px1, 0);        ix2 = min(px2, w)
        
        if iy2 <= iy1 or ix2 <= ix1:
            continue
        
        vis = (iy2 - iy1) * (ix2 - ix1)
        orig = (py2 - py1) * (px2 - px1)
        
        # Lower visibility threshold for person (20%), normal for others (50%)
        min_vis = 0.2 if cls == 0 else 0.5
        if orig > 0 and vis / orig < min_vis:
            continue
        
        # Normalize relative to crop, then scale back since we resized to original
        new_cx = (ix1 + ix2) / 2 / w
        new_cy = ((iy1 + iy2) / 2 - crop_y1) / new_h
        new_bw = (ix2 - ix1) / w
        new_bh = (iy2 - iy1) / new_h
        
        if is_valid_box(new_cx, new_cy, new_bw, new_bh):
            new_boxes.append([cls, new_cx, new_cy, new_bw, new_bh])
    
    return cropped, new_boxes


augmentations = {'crop': random_crop,
                 'flip': horizontal_flip,
                 'crop person': cropped_person}


# ~~~~~~~~~~~ Main loop to process dataset ~~~~~~~~~~~~

img_paths = list(Path(IMG_DIR).glob("*.jpg"))

for img_path in img_paths:
    label_path = Path(LABEL_DIR) / (img_path.stem + ".txt")
    if not label_path.exists():
        continue
    
    labels = load_labels(str(label_path))
        
    if not labels:
        continue  # Skip images without target classes
    
    img = cv2.imread(str(img_path))
    if img is None:
        continue

    for aug_name, aug_func in augmentations.items():
        aug_img, aug_labels = aug_func(img.copy(), [list(b) for b in labels])
        
        if aug_img is None or not aug_labels:
            continue  # this augmentation didn't apply to this image
        
        out_stem = f"{img_path.stem}_{aug_name}"
        cv2.imwrite(str(Path(OUT_IMG) / f"{out_stem}.jpg"), aug_img)
        save_labels(aug_labels, str(Path(OUT_LBL) / f"{out_stem}.txt"))
           

: 

In [ ]:
# ~~~~~~~~~~~~ Statistics ad Visualizations ~~~~~~~~~~~~
total_augmented = len(list(Path(OUT_IMG).glob("*.jpg")))
print(f"✓ Augmentation complete: {total_augmented} augmented images created in {OUT_IMG}")
total_augmented_labels = len(list(Path(OUT_LBL).glob("*.txt")))
print(f"✓ Augmentation complete: {total_augmented_labels} augmented label files created in {OUT_LBL}")

import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pathlib import Path
from collections import Counter, defaultdict

# ~~~~~~~~~~~ Config ~~~~~~~~~~~~
ROOT       = r"C:\Users\youch\CPP_AVL_ObjDet"
LABEL_DIRS = {
    "train":           ROOT + r"\labels\train",
    "train_augmented": ROOT + r"\labels\train_augmented",
}
YAML_PATH  = ROOT + r"\config.yml"

with open(YAML_PATH) as f:
    cfg = yaml.safe_load(f)
names = cfg["names"]  # ['person', 'bike', 'car', ...]


# ~~~~~~~~~~~ Load all labels from a folder ~~~~~~~~~~~~
def load_all_labels(label_dir):
    """Returns list of (class_id, cx, cy, w, h) for all boxes in folder"""
    all_boxes = []
    for txt in Path(label_dir).glob("*.txt"):
        with open(txt) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    cls = int(parts[0])
                    cx, cy, w, h = map(float, parts[1:])
                    all_boxes.append((cls, cx, cy, w, h))
    return all_boxes


# ~~~~~~~~~~~ Gather stats ~~~~~~~~~~~~
stats = {}
for split, ldir in LABEL_DIRS.items():
    boxes = load_all_labels(ldir)
    stats[split] = {
        "boxes":       boxes,
        "num_images":  len(list(Path(ldir).glob("*.txt"))),
        "num_boxes":   len(boxes),
        "class_count": Counter(b[0] for b in boxes),
        "widths":      [b[3] for b in boxes],
        "heights":     [b[4] for b in boxes],
        "areas":       [b[3] * b[4] for b in boxes],
    }


# ~~~~~~~~~~~ Plot ~~~~~~~~~~~~
fig = plt.figure(figsize=(18, 14))
fig.suptitle("Dataset Statistics: train vs train_augmented", fontsize=15, fontweight="bold")
gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.45, wspace=0.35)

colors = {"train": "#4C72B0", "train_augmented": "#DD8452"}

# ── 1. Image & box count ──────────────────────────────────────────
ax1 = fig.add_subplot(gs[0, 0])
splits   = list(stats.keys())
n_images = [stats[s]["num_images"] for s in splits]
n_boxes  = [stats[s]["num_boxes"]  for s in splits]
x = np.arange(len(splits))
bars = ax1.bar(x - 0.2, n_images, 0.35, label="Images", color=[colors[s] for s in splits], alpha=0.85)
bars2 = ax1.bar(x + 0.2, n_boxes,  0.35, label="Boxes",  color=[colors[s] for s in splits], alpha=0.5)
ax1.set_xticks(x); ax1.set_xticklabels(splits, rotation=10)
ax1.set_title("Image & Box Count")
ax1.legend(fontsize=8)
for bar in list(bars) + list(bars2):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
             str(int(bar.get_height())), ha='center', va='bottom', fontsize=8)


# ── 2. Class distribution per split ──────────────────────────────
ax2 = fig.add_subplot(gs[0, 1:])
all_class_ids = sorted(set(
    cid for s in stats for cid in stats[s]["class_count"]
))
class_labels = [names[i] if i < len(names) else str(i) for i in all_class_ids]
x = np.arange(len(all_class_ids))
width = 0.35
for i, (split, color) in enumerate(colors.items()):
    counts = [stats[split]["class_count"].get(cid, 0) for cid in all_class_ids]
    offset = (i - 0.5) * width
    bars = ax2.bar(x + offset, counts, width, label=split, color=color, alpha=0.85)
ax2.set_xticks(x)
ax2.set_xticklabels(class_labels, rotation=35, ha='right', fontsize=8)
ax2.set_title("Class Distribution")
ax2.legend(fontsize=8)


# ── 3. Box width distribution ─────────────────────────────────────
ax3 = fig.add_subplot(gs[1, 0])
for split, color in colors.items():
    ax3.hist(stats[split]["widths"], bins=50, color=color,
             alpha=0.6, label=split, density=True)
ax3.set_title("Box Width Distribution (normalized)")
ax3.set_xlabel("Width (0–1)"); ax3.legend(fontsize=8)


# ── 4. Box height distribution ────────────────────────────────────
ax4 = fig.add_subplot(gs[1, 1])
for split, color in colors.items():
    ax4.hist(stats[split]["heights"], bins=50, color=color,
             alpha=0.6, label=split, density=True)
ax4.set_title("Box Height Distribution (normalized)")
ax4.set_xlabel("Height (0–1)"); ax4.legend(fontsize=8)


# ── 5. Box area distribution ──────────────────────────────────────
ax5 = fig.add_subplot(gs[1, 2])
for split, color in colors.items():
    ax5.hist(stats[split]["areas"], bins=50, color=color,
             alpha=0.6, label=split, density=True)
ax5.set_title("Box Area Distribution (normalized)")
ax5.set_xlabel("Area (w×h)"); ax5.legend(fontsize=8)


# ── 6. Box center scatter (cx, cy) ───────────────────────────────
for i, (split, color) in enumerate(colors.items()):
    ax = fig.add_subplot(gs[2, i])
    cxs = [b[1] for b in stats[split]["boxes"]]
    cys = [b[2] for b in stats[split]["boxes"]]
    ax.scatter(cxs, cys, s=1, alpha=0.3, color=color)
    ax.set_xlim(0, 1); ax.set_ylim(0, 1)
    ax.invert_yaxis()
    ax.set_title(f"Box Centers: {split}")
    ax.set_xlabel("cx"); ax.set_ylabel("cy")


# ── 7. Summary table ─────────────────────────────────────────────
ax7 = fig.add_subplot(gs[2, 2])
ax7.axis("off")
table_data = []
for split in splits:
    s = stats[split]
    avg_per_img = s["num_boxes"] / s["num_images"] if s["num_images"] > 0 else 0
    table_data.append([
        split,
        str(s["num_images"]),
        str(s["num_boxes"]),
        f"{avg_per_img:.1f}",
    ])
table = ax7.table(
    cellText=table_data,
    colLabels=["Split", "Images", "Boxes", "Avg boxes/img"],
    loc="center", cellLoc="center"
)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 1.8)
ax7.set_title("Summary", fontweight="bold")

plt.savefig(str(Path(ROOT) / "augmentation_stats.png"), dpi=150, bbox_inches="tight")
plt.show()
print(f"Stats saved to {ROOT}\\augmentation_stats.png")
    


✓ Augmentation complete: 17155 augmented images created in C:\Users\youch\CPP_AVL_ObjDet\images\train_augmented
✓ Augmentation complete: 17155 augmented label files created in C:\Users\youch\CPP_AVL_ObjDet\labels\train_augmented
